In [49]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import Session
import json
import sagemaker
import yt_dlp
from moviepy.video.io.VideoFileClip import VideoFileClip
import boto3
import os

In [50]:
session = Session()
instance_type = "ml.g5.2xlarge"
bucket = 'slip-ml'
role = 'arn:aws:iam::438465160412:role/Sagemaker'

In [51]:
image_uri = sagemaker.image_uris.retrieve(framework='pytorch',
                             region=session.boto_region_name,
                             instance_type=instance_type,
                             image_scope='inference')
print(image_uri)

[05/15/25 14:09:28] INFO     Defaulting to only available Python version: py312                   ]8;id=877876;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=260233;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/image_uris.py#610\610]8;;\

763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:2.6.0-gpu-py312


In [52]:
one_model = "s3://slip-ml/models/vallr-transformer/pytorch-training-2025-05-14-23-01-24-338/output/model.tar.gz"

In [53]:
model = PyTorchModel(
    model_data=one_model,  # Dummy, as models are loaded from S3 in serve.py
    role=role,
    entry_point="serve.py",
    source_dir="source",
    image_uri=image_uri,
    sagemaker_session=session,
    env={
        "SM_DEFAULT_BUCKET": f"{bucket}",
        "VALLR_S3_KEY": one_model,
        "LLAMA_S3_KEY": "models/llama_model.tar.gz",
    }
)

In [54]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,  # Single-GPU
    endpoint_name="vallr-llama-endpoint"
)

[05/15/25 14:09:33] INFO     Repacking model artifact                                                  ]8;id=197927;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=702529;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/model.py#820\820]8;;\
                             (s3://slip-ml/models/vallr-transformer/pytorch-training-2025-05-14-23-01-             
                             24-338/output/model.tar.gz), script artifact (source), and dependencies               
                             ([]) into single tar.gz file located at                                               
                             s3://sagemaker-us-east-1-438465160412/pytorch-inference-2025-05-15-20-09-             
                             32-910/model.tar.gz. This may take some time depending on model size...               

[05/15/25 14:10:51] INFO     Creating model with name: pytorch-inference-2025-05-15-20-10-51-133    ]8;id=252149;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=62995;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name vallr-llama-endpoint                ]8;id=747831;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=453181;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py#6019\6019]8;;\

[05/15/25 14:10:52] INFO     Creating endpoint with name vallr-llama-endpoint                       ]8;id=425204;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=442842;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py#4841\4841]8;;\

--------------!

In [44]:
def download_youtube_video_yt_dlp(url):
    # extract video ID from the URL
    video_id = url.split("v=")[-1]
    if "&" in video_id:
        video_id = video_id.split("&")[0]

    ydl_opts = {
        "outtmpl": f"{video_id}.%(ext)s",  # Output path and filename
        "format": "best",  # Select the best single file (video + audio)
        "merge_output_format": None,  # Avoid merging, stick to single stream
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("Download completed successfully!")
        return f"{video_id}.mp4"
    except Exception as e:
        print(f"An error occurred: {e}")

In [45]:
youtube_test = 'https://www.youtube.com/watch?v=ePRrIb1NCkw'
video_file = download_youtube_video_yt_dlp(youtube_test)
video = VideoFileClip(video_file)
chunk = video.subclipped(30, 40)
chunk_filename = f"testing_vid.mp4"
chunk.write_videofile(chunk_filename, codec="libx264")
# upload the video to S3
s3 = boto3.client('s3')
s3.upload_file(chunk_filename, bucket, chunk_filename)
# delete the local video file
os.remove(video_file)

[youtube] Extracting URL: https://www.youtube.com/watch?v=ePRrIb1NCkw
[youtube] ePRrIb1NCkw: Downloading webpage
[youtube] ePRrIb1NCkw: Downloading tv client config
[youtube] ePRrIb1NCkw: Downloading player b2858d36-main
[youtube] ePRrIb1NCkw: Downloading tv player API JSON
[youtube] ePRrIb1NCkw: Downloading ios player API JSON
[youtube] ePRrIb1NCkw: Downloading m3u8 information
[info] ePRrIb1NCkw: Downloading 1 format(s): 18
[download] Destination: ePRrIb1NCkw.mp4
[download] 100% of   16.64MiB in 00:00:01 at 10.25MiB/s  
Download completed successfully!
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'encoder': 'Google'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 360], 'bitrate': 161, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media

MoviePy - Done.
MoviePy - Writing video testing_vid.mp4



MoviePy - Done !
MoviePy - video ready testing_vid.mp4


In [47]:
response = predictor.predict({
    "input_s3_path": f"s3://{bucket}/{chunk_filename}",
})
print(json.loads(response))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 response = predictor.predict({                                                               │
│   2 │   "input_s3_path": f"s3://{bucket}/{chunk_filename}",                                      │
│   3 })                                                                                           │
│   4 print(json.loads(response))                                                                  │
│                                                                                                  │
│ /Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagem │
│ aker/base_predictor.py:212 in predict                                                            │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/botoc │
│ ore/client.py:570 in _api_call                                                                   │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/botoc │
│ ore/context.py:123 in wrapper                                                                    │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                      